In [3]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.1913
epoch [2/100], loss:0.1606
epoch [3/100], loss:0.1655
epoch [4/100], loss:0.1600
epoch [5/100], loss:0.1508
epoch [6/100], loss:0.1417
epoch [7/100], loss:0.1388
epoch [8/100], loss:0.1382
epoch [9/100], loss:0.1380
epoch [10/100], loss:0.1516
epoch [11/100], loss:0.1418
epoch [12/100], loss:0.1390
epoch [13/100], loss:0.1522
epoch [14/100], loss:0.1457
epoch [15/100], loss:0.1250
epoch [16/100], loss:0.1433
epoch [17/100], loss:0.1328
epoch [18/100], loss:0.1351
epoch [19/100], loss:0.1263
epoch [20/100], loss:0.1365
epoch [21/100], loss:0.1293
epoch [22/100], loss:0.1301
epoch [23/100], loss:0.1181
epoch [24/100], loss:0.1354
epoch [25/100], loss:0.1208
epoch [26/100], loss:0.1295
epoch [27/100], loss:0.1294
epoch [28/100], loss:0.1173
epoch [29/100], loss:0.1167
epoch [30/100], loss:0.1303
epoch [31/100], loss:0.1336
epoch [32/100], loss:0.1242
epoch [33/100], loss:0.1345
epoch [34/100], loss:0.1298
epoch [35/100], loss:0.1234
epoch [36/100], loss:0.1295
e

## Import Libraries

In [3]:
import os
import torch
import torchvision

from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [15]:
import torch
from torchvision.transforms import transforms
from PIL import Image
from pathlib import Path

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = autoencoder().cuda()
checkpoint = torch.load('sim_autoencoder.pth')
model.load_state_dict(checkpoint)
trans = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    ])

image = Image.open('panda.jpg')

input = trans(image)

input = input.view(1, 3, 32,32)

output = model(input)

prediction = int(torch.max(output.data, 1)[1].numpy())
print(prediction)


RuntimeError: Expected object of backend CPU but got backend CUDA for argument #2 'mat2'